In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    for filename in filenames:
        break
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt pycocotools # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/
Image(filename='runs/detect/exp/zidane.jpg', width=600)

In [ ]:
!rm -rf yolo_data
!mkdir -p yolo_data/images/train
!mkdir -p yolo_data/labels/train

!cp /kaggle/input/nfl-impact-detection/images/*.jpg yolo_data/images/train
!cp /kaggle/input/nfl-helmet-single-class/yolo_train_data/*.txt yolo_data/labels/train

!ls yolo_data/images/train/*.jpg | wc
!ls yolo_data/labels/train/*.txt | wc

In [ ]:
%%writefile nfl.yaml
# https://raw.githubusercontent.com/ultralytics/yolov5/master/data/coco128.yaml
train: yolo_data/images/train/  # 9947 images
val: yolo_data/images/train/  # 9947 images

# number of classes
nc: 1

# class names
names: ['Helmet']

In [ ]:
!cat nfl.yaml

In [ ]:
# torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip', 'tmp.zip')
# !unzip -q tmp.zip -d ../ && rm tmp.zip
# !WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 3 --data nfl.yaml --weights yolov5s.pt --cache 

In [ ]:
# !WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache 
!WANDB_MODE="dryrun" python train.py --img 640 --batch 16 --epochs 15 --data nfl.yaml --weights yolov5s.pt 

In [ ]:
!rm -rf yolo_data/images
!rm -rf yolo_data/labels
!cp -r runs/train/exp/weights/ .

In [ ]:
!cp -r yolov5/runs/train/exp/weights/ yolov5s_weights